In [ ]:
import numpy as np

from sklearn import datasets
from sklearn.manifold import TSNE

import plotly.plotly as py
import plotly.graph_objs as go

import plotly

# Free users can only have 50 plots. So everytime we run out of plot for one user, create another one
# And paste it here.
plotly.tools.set_credentials_file(username='wBIr68ns', api_key='LAK0vePuQsXlQQFYaKJv')


# 3 dimensions
ndims = 3
scatterClass = go.Scatter3d
if ndims == 2:
    scatterClass = go.Scatter
    
# Label of the plot
plot_title = 'Visualise'

In [ ]:
def get_mnist_dataset():
    digits = datasets.load_digits()
    data = digits.data[:500]
    labels = digits.target[:500]
    return data, labels


def convert_mnist_labels_to_string(labels):
    return np.array([str(label) for label in labels])


def perform_tsne(data):    
    tsne = TSNE(n_components=ndims, random_state=0)
    return tsne.fit_transform(data)


In [ ]:
data, labels = get_mnist_dataset()
labels = convert_mnist_labels_to_string(labels)
unique_labels = np.unique(labels)
data_tsne = perform_tsne(data)

In [ ]:
traces = []
for label in unique_labels:
    ind = np.where(labels==label)
    coords = dict(
        x=data_tsne[ind, 0].ravel(),
        y=data_tsne[ind, 1].ravel()
    )
    if ndims == 3:
        coords['z'] = data_tsne[ind, 2].ravel()

    ids = labels[ind];

    trace = scatterClass(
        name=label.strip(),
        mode='markers',
        marker=dict(
            size=5,
            line=dict(
                width=0.5
            ),
            opacity=1
        ),
        text=ids,
        **coords
    )
    traces.append(trace)
layout = go.Layout(
    hovermode='closest',
    title=plot_title,
    margin=dict(
        l=0,
        r=0,
        b=0,
        t=0
    )
)
fig = go.Figure(data=traces, layout=layout)
plot = py.iplot(fig, filename=plot_title)
print('{}: {}'.format(plot_title, plot.resource))